<a href="https://colab.research.google.com/github/UmaBalannavar/CNS-ASSINMENT/blob/main/RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# rsa_demo.py
# Educational RSA implementation (Miller-Rabin primality, keygen, encrypt/decrypt, sign/verify)
import random
import hashlib

# ---------- Helpers ----------
def egcd(a, b):
    if b == 0:
        return (a, 1, 0)
    g, x1, y1 = egcd(b, a % b)
    return (g, y1, x1 - (a // b) * y1)

def modinv(a, m):
    g, x, _ = egcd(a % m, m)
    if g != 1:
        raise ValueError("No modular inverse")
    return x % m

def is_prime(n, k=10):
    """Miller-Rabin primality test"""
    if n < 2:
        return False
    small_primes = [2,3,5,7,11,13,17,19,23,29]
    for p in small_primes:
        if n % p == 0:
            return n == p
    # write n-1 as d * 2^s
    d = n - 1
    s = 0
    while d % 2 == 0:
        d //= 2
        s += 1
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for __ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def gen_prime(bits):
    while True:
        p = random.getrandbits(bits) | (1 << (bits - 1)) | 1
        if is_prime(p):
            return p

# ---------- Key generation ----------
def rsa_keygen(bits=1024):
    # For demo you can set bits=512 or 256 (faster) but insecure.
    p = gen_prime(bits//2)
    q = gen_prime(bits//2)
    while q == p:
        q = gen_prime(bits//2)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    if phi % e == 0:
        # choose another e
        e = 3
    d = modinv(e, phi)
    public = (n, e)
    private = (n, d)
    return public, private

# ---------- Encryption / Decryption ----------
def rsa_encrypt_int(m_int, pub):
    n, e = pub
    if m_int >= n:
        raise ValueError("Plaintext too large")
    return pow(m_int, e, n)

def rsa_decrypt_int(c_int, priv):
    n, d = priv
    return pow(c_int, d, n)

# convenience for text
def text_to_int(msg: bytes):
    return int.from_bytes(msg, byteorder='big')

def int_to_text(m_int: int):
    length = (m_int.bit_length() + 7) // 8
    return m_int.to_bytes(length, byteorder='big')

# ---------- Sign / Verify (RSA PKCS#1-like simple) ----------
def rsa_sign(message_bytes: bytes, priv):
    # Hash then sign (simple) -- not PKCS#1 padded properly, educational only
    h = int.from_bytes(hashlib.sha256(message_bytes).digest(), 'big')
    return rsa_decrypt_int(h, priv)  # sign using private exponent (m = h^d mod n)

def rsa_verify(message_bytes: bytes, signature_int: int, pub):
    h = int.from_bytes(hashlib.sha256(message_bytes).digest(), 'big')
    recovered = rsa_encrypt_int(signature_int, pub)
    return recovered == h

# ---------- Example usage ----------
if __name__ == "__main__":
    print("Generating RSA keypair (demo, 512 bits)...")
    pub, priv = rsa_keygen(bits=512)  # faster for demo; increase bits for more security
    message = b'Hello RSA!'
    m_int = text_to_int(message)
    c = rsa_encrypt_int(m_int, pub)
    print("Ciphertext (int):", c)
    m2 = rsa_decrypt_int(c, priv)
    print("Recovered message:", int_to_text(m2))

    # Signing
    sig = rsa_sign(message, priv)
    print("Signature (int):", sig)
    print("Signature valid?", rsa_verify(message, sig, pub))


Generating RSA keypair (demo, 512 bits)...
Ciphertext (int): 1481732960162872965994875082320161415542209483115410124106733148097831956839063068241957372873361171523409986200836911212906167190819056696958225735544272
Recovered message: b'Hello RSA!'
Signature (int): 5747800890687793452587760139605259665388680005221076041253327783261165924578652372324648509466864744064890814201777104311418528893997997334297232657774048
Signature valid? True


In [ ]:
# elgamal_demo.py
# Educational ElGamal over prime field (text -> int mapping)
import random
import hashlib

def egcd(a, b):
    if b == 0:
        return (a, 1, 0)
    g, x1, y1 = egcd(b, a % b)
    return (g, y1, x1 - (a // b) * y1)

def modinv(a, m):
    g, x, _ = egcd(a % m, m)
    if g != 1:
        raise ValueError("No modular inverse")
    return x % m

def is_prime(n, k=8):
    if n < 2:
        return False
    small = [2,3,5,7,11,13,17,19]
    for p in small:
        if n % p == 0:
            return n == p
    # Miller-Rabin (short)
    d = n - 1
    s = 0
    while d % 2 == 0:
        s += 1
        d //= 2
    for _ in range(k):
        a = random.randrange(2, n-1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for __ in range(s-1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def gen_safe_prime(bits):
    while True:
        p = random.getrandbits(bits) | (1 << (bits-1)) | 1
        if is_prime(p):
            return p

def find_primitive_root(p):
    # naive approach: find g with order p-1
    if p == 2:
        return 1
    phi = p - 1
    # factor phi (simple)
    factors = set()
    n = phi
    d = 2
    while d * d <= n:
        if n % d == 0:
            factors.add(d)
            while n % d == 0:
                n //= d
        d += 1
    if n > 1:
        factors.add(n)
    for g in range(2, p):
        ok = True
        for q in factors:
            if pow(g, phi // q, p) == 1:
                ok = False
                break
        if ok:
            return g
    raise ValueError("No primitive root found")

# ---------- Keygen ----------
def elgamal_keygen(bits=256):
    p = gen_safe_prime(bits)
    g = find_primitive_root(p)
    x = random.randrange(2, p-2)  # private
    h = pow(g, x, p)
    return (p, g, h), x  # public, private

# ---------- Encryption / Decryption ----------
def elgamal_encrypt_bytes(plaintext: bytes, pub):
    p, g, h = pub
    m = int.from_bytes(plaintext, 'big')
    if m >= p:
        raise ValueError("Message too large for chosen prime p")
    y = random.randrange(1, p-1)
    c1 = pow(g, y, p)
    s = pow(h, y, p)
    c2 = (m * s) % p
    return (c1, c2)

def elgamal_decrypt_bytes(cipherpair, priv, pub):
    p, g, h = pub
    c1, c2 = cipherpair
    s = pow(c1, priv, p)
    m = (c2 * modinv(s, p)) % p
    # convert back to bytes
    length = (m.bit_length() + 7) // 8
    return m.to_bytes(length, 'big')

# ---------- Example ----------
if __name__ == "__main__":
    print("Generating ElGamal keys (demo, 256 bits prime)...")
    pub, priv = elgamal_keygen(bits=256)
    p, g, h = pub
    msg = b'Hi ElGamal'
    ct = elgamal_encrypt_bytes(msg, pub)
    print("Cipher pair:", ct)
    pt = elgamal_decrypt_bytes(ct, priv, pub)
    print("Recovered:", pt)


Generating ElGamal keys (demo, 256 bits prime)...


KeyboardInterrupt: 

In [ ]:
# ecc_ecdsa_demo.py
# Educational ECDSA implementation (secp256k1) for keygen, sign, verify.
# WARNING: This is educational. For production use well-tested libs.

import random
import hashlib

# secp256k1 domain parameters
P = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
A = 0
B = 7
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

# Point at infinity representation
INF = None

def inv_mod(x, p):
    # modular inverse
    return pow(x, p-2, p)

def is_on_curve(point):
    if point is None:
        return True
    x, y = point
    return (y*y - (x*x*x + A*x + B)) % P == 0

def point_add(p1, p2):
    if p1 is None:
        return p2
    if p2 is None:
        return p1
    x1, y1 = p1
    x2, y2 = p2
    if x1 == x2 and (y1 != y2 or y1 == 0):
        return None
    if x1 == x2:
        # point doubling
        m = (3 * x1 * x1 + A) * inv_mod(2 * y1, P) % P
    else:
        m = (y2 - y1) * inv_mod(x2 - x1, P) % P
    x3 = (m*m - x1 - x2) % P
    y3 = (m*(x1 - x3) - y1) % P
    return (x3, y3)

def scalar_mult(k, point):
    # double-and-add
    result = None
    addend = point
    while k:
        if k & 1:
            result = point_add(result, addend)
        addend = point_add(addend, addend)
        k >>= 1
    return result

# ---------- Key generation ----------
def ecc_keygen():
    priv = random.randrange(1, N-1)
    pub = scalar_mult(priv, (Gx, Gy))
    return priv, pub

# ---------- ECDSA sign / verify ----------
def ecc_sign(msg: bytes, priv):
    z = int.from_bytes(hashlib.sha256(msg).digest(), 'big')
    while True:
        k = random.randrange(1, N-1)
        x1y1 = scalar_mult(k, (Gx, Gy))
        if x1y1 is None:
            continue
        r = x1y1[0] % N
        if r == 0:
            continue
        s = (inv_mod(k, N) * (z + r * priv)) % N
        if s == 0:
            continue
        return (r, s)

def ecc_verify(msg: bytes, signature, pub):
    r, s = signature
    if not (1 <= r < N and 1 <= s < N):
        return False
    z = int.from_bytes(hashlib.sha256(msg).digest(), 'big')
    w = inv_mod(s, N)
    u1 = (z * w) % N
    u2 = (r * w) % N
    p = point_add(scalar_mult(u1, (Gx, Gy)), scalar_mult(u2, pub))
    if p is None:
        return False
    return (p[0] % N) == r

# ---------- Example ----------
if __name__ == "__main__":
    print("Generating ECC keypair (secp256k1)...")
    priv, pub = ecc_keygen()
    print("Private key (int):", priv)
    print("Public key (point):", pub)
    message = b'Hello ECC ECDSA!'
    sig = ecc_sign(message, priv)
    print("Signature (r,s):", sig)
    print("Verify:", ecc_verify(message, sig, pub))
